### 1. Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [4]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

### 2.  Data processing

### 2.1 read in data

In [5]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [6]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
dataset.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

### 2.2 Feature selection: excluding columns of 'RowNumber', 'CustomerId', 'Surname'

In [8]:
X = dataset.iloc[:, 3:13].values

In [9]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [10]:
y = dataset.iloc[:, 13].values

In [11]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

### 2.3 Categorical encoding: column Geography and Gender

In [12]:
#Label encoder for ''Geography'', 1 is the index of 'country' col
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])

In [13]:
X

array([[619, 0, 'Female', ..., 1, 1, 101348.88],
       [608, 2, 'Female', ..., 0, 1, 112542.58],
       [502, 0, 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 0, 'Female', ..., 0, 1, 42085.58],
       [772, 1, 'Male', ..., 1, 0, 92888.52],
       [792, 0, 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [14]:
#Label encoder for 'Gender'
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

#### remove genography column

In [15]:
X_temp = np.delete(X, 1, 1)
X_temp

array([[619, 0, 42, ..., 1, 1, 101348.88],
       [608, 0, 41, ..., 0, 1, 112542.58],
       [502, 0, 42, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 36, ..., 0, 1, 42085.58],
       [772, 1, 42, ..., 1, 0, 92888.52],
       [792, 0, 28, ..., 1, 0, 38190.78]], dtype=object)

### 2.4 One-hot encoding: column Geography 

In [16]:
#One-hot encoding ''Geography'' col
#One-hot encoding 'country' col
onehotencoder = OneHotEncoder()
geography_array = onehotencoder.fit_transform(X[:, 1:2]).toarray()

### 2.5 avoid dummy data trap

In [17]:
geography_array = geography_array[: , 1:3]
geography_array

array([[0., 0.],
       [0., 1.],
       [0., 0.],
       ...,
       [0., 0.],
       [1., 0.],
       [0., 0.]])

In [18]:
X_final = np.concatenate( (geography_array, X_temp),  axis = 1)
X_final

array([[0.0, 0.0, 619, ..., 1, 1, 101348.88],
       [0.0, 1.0, 608, ..., 0, 1, 112542.58],
       [0.0, 0.0, 502, ..., 1, 0, 113931.57],
       ...,
       [0.0, 0.0, 709, ..., 0, 1, 42085.58],
       [1.0, 0.0, 772, ..., 1, 0, 92888.52],
       [0.0, 0.0, 792, ..., 1, 0, 38190.78]], dtype=object)

### 2.6 Data split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size = 0.2, random_state = 0)

### 2.7 Feature scaling

In [20]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### 3. K-fold cross validation

In [21]:
##============evalate the model using k-fold cross validatioN
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [22]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 50)

In [23]:
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/50
7200/7200 [==============================] - 2s 313us/step - loss: 0.4814 - acc: 0.7969
Epoch 2/50
7200/7200 [==============================] - ETA: 0s - loss: 0.4285 - acc: 0.797 - 2s 258us/step - loss: 0.4292 - acc: 0.7971
Epoch 3/50
7200/7200 [==============================] - 1s 174us/step - loss: 0.4241 - acc: 0.7971
Epoch 4/50
7200/7200 [==============================] - 1s 163us/step - loss: 0.4195 - acc: 0.8140
Epoch 5/50
7200/7200 [==============================] - 1s 159us/step - loss: 0.4171 - acc: 0.8219
Epoch 6/50
7200/7200 [==============================] - 1s 161us/step - loss: 0.4150 - acc: 0.8289
Epoch 7/50
7200/7200 [==============================] - 1s 202us/step - loss: 0.4130 - acc: 0.8319
Epoch 8/50
7200/7200 [==============================] - 1s 189us/step - loss: 0.4121 - acc: 0.8310
Epoch 9/50
7200/7200 [==============================] - 1s 153us/step 

7200/7200 [==============================] - 2s 256us/step - loss: 0.4086 - acc: 0.8317
Epoch 15/50
7200/7200 [==============================] - 1s 189us/step - loss: 0.4077 - acc: 0.8311
Epoch 16/50
7200/7200 [==============================] - 2s 242us/step - loss: 0.4071 - acc: 0.8333
Epoch 17/50
7200/7200 [==============================] - 2s 239us/step - loss: 0.4057 - acc: 0.8361
Epoch 18/50
7200/7200 [==============================] - 2s 217us/step - loss: 0.4047 - acc: 0.8339
Epoch 19/50
7200/7200 [==============================] - 2s 227us/step - loss: 0.4035 - acc: 0.8333
Epoch 20/50
7200/7200 [==============================] - 1s 177us/step - loss: 0.4029 - acc: 0.8354
Epoch 21/50
7200/7200 [==============================] - 1s 195us/step - loss: 0.4021 - acc: 0.8358
Epoch 22/50
7200/7200 [==============================] - 1s 173us/step - loss: 0.4012 - acc: 0.8350
Epoch 23/50
7200/7200 [==============================] - 1s 177us/step - loss: 0.4004 - acc: 0.8336
Epoch 24/50


7200/7200 [==============================] - 1s 163us/step - loss: 0.4023 - acc: 0.8346
Epoch 46/50
7200/7200 [==============================] - 1s 158us/step - loss: 0.4017 - acc: 0.8350
Epoch 47/50
7200/7200 [==============================] - 1s 190us/step - loss: 0.4020 - acc: 0.8343 1s - 
Epoch 48/50
7200/7200 [==============================] - 1s 178us/step - loss: 0.4011 - acc: 0.8346
Epoch 49/50
7200/7200 [==============================] - 1s 169us/step - loss: 0.4012 - acc: 0.8353
Epoch 50/50
800/800 [==============================] - 0s 276us/step
Epoch 1/50
7200/7200 [==============================] - 2s 279us/step - loss: 0.4961 - acc: 0.7964
Epoch 2/50
7200/7200 [==============================] - 1s 187us/step - loss: 0.4273 - acc: 0.7975
Epoch 3/50
7200/7200 [==============================] - 1s 181us/step - loss: 0.4230 - acc: 0.7975
Epoch 4/50
7200/7200 [==============================] - 1s 165us/step - loss: 0.4194 - acc: 0.7975
Epoch 5/50
7200/7200 [===================

7200/7200 [==============================] - 1s 118us/step - loss: 0.3989 - acc: 0.8350
Epoch 26/50
7200/7200 [==============================] - 1s 116us/step - loss: 0.3990 - acc: 0.8342
Epoch 27/50
7200/7200 [==============================] - 1s 146us/step - loss: 0.3979 - acc: 0.8346
Epoch 28/50
7200/7200 [==============================] - 1s 151us/step - loss: 0.3972 - acc: 0.8357
Epoch 29/50
7200/7200 [==============================] - 1s 116us/step - loss: 0.3980 - acc: 0.8358
Epoch 30/50
7200/7200 [==============================] - 1s 118us/step - loss: 0.3978 - acc: 0.8347
Epoch 31/50
7200/7200 [==============================] - 1s 109us/step - loss: 0.3979 - acc: 0.8343
Epoch 32/50
7200/7200 [==============================] - 1s 123us/step - loss: 0.3974 - acc: 0.8350
Epoch 33/50
7200/7200 [==============================] - 1s 115us/step - loss: 0.3970 - acc: 0.8371
Epoch 34/50
7200/7200 [==============================] - 1s 123us/step - loss: 0.3968 - acc: 0.8353
Epoch 35/50


7200/7200 [==============================] - 1s 124us/step - loss: 0.4179 - acc: 0.8237
Epoch 6/50
7200/7200 [==============================] - 1s 127us/step - loss: 0.4158 - acc: 0.8271
Epoch 7/50
7200/7200 [==============================] - 1s 122us/step - loss: 0.4140 - acc: 0.8300
Epoch 8/50
7200/7200 [==============================] - 1s 115us/step - loss: 0.4134 - acc: 0.8310
Epoch 9/50
7200/7200 [==============================] - 1s 115us/step - loss: 0.4126 - acc: 0.8328
Epoch 10/50
7200/7200 [==============================] - 1s 113us/step - loss: 0.4114 - acc: 0.8337
Epoch 11/50
7200/7200 [==============================] - 1s 128us/step - loss: 0.4103 - acc: 0.8339
Epoch 12/50
7200/7200 [==============================] - 1s 149us/step - loss: 0.4098 - acc: 0.8356
Epoch 13/50
7200/7200 [==============================] - 1s 138us/step - loss: 0.4092 - acc: 0.8358
Epoch 14/50
7200/7200 [==============================] - 1s 143us/step - loss: 0.4083 - acc: 0.8353
Epoch 15/50
7200

7200/7200 [==============================] - 1s 174us/step - loss: 0.3931 - acc: 0.8365
Epoch 37/50
7200/7200 [==============================] - 1s 163us/step - loss: 0.3930 - acc: 0.8379
Epoch 38/50
7200/7200 [==============================] - 1s 146us/step - loss: 0.3932 - acc: 0.8357
Epoch 39/50
7200/7200 [==============================] - 1s 180us/step - loss: 0.3925 - acc: 0.8376
Epoch 40/50
7200/7200 [==============================] - 1s 200us/step - loss: 0.3924 - acc: 0.8368
Epoch 41/50
7200/7200 [==============================] - 1s 153us/step - loss: 0.3924 - acc: 0.8387
Epoch 42/50
7200/7200 [==============================] - 1s 143us/step - loss: 0.3927 - acc: 0.8381
Epoch 43/50
7200/7200 [==============================] - 1s 166us/step - loss: 0.3922 - acc: 0.8379
Epoch 44/50
7200/7200 [==============================] - 1s 160us/step - loss: 0.3929 - acc: 0.8372
Epoch 45/50
7200/7200 [==============================] - 1s 176us/step - loss: 0.3925 - acc: 0.8357
Epoch 46/50


7200/7200 [==============================] - 1s 179us/step - loss: 0.4010 - acc: 0.8336
Epoch 17/50
7200/7200 [==============================] - 2s 225us/step - loss: 0.4009 - acc: 0.8340
Epoch 18/50
7200/7200 [==============================] - 1s 141us/step - loss: 0.4007 - acc: 0.8347
Epoch 19/50
7200/7200 [==============================] - 1s 154us/step - loss: 0.4005 - acc: 0.8347
Epoch 20/50
7200/7200 [==============================] - 1s 143us/step - loss: 0.3996 - acc: 0.8343
Epoch 21/50
7200/7200 [==============================] - 1s 160us/step - loss: 0.4001 - acc: 0.8329
Epoch 22/50
7200/7200 [==============================] - 1s 184us/step - loss: 0.3995 - acc: 0.8350
Epoch 23/50
7200/7200 [==============================] - 1s 163us/step - loss: 0.3996 - acc: 0.8333
Epoch 24/50
7200/7200 [==============================] - 1s 182us/step - loss: 0.3991 - acc: 0.8346
Epoch 25/50
7200/7200 [==============================] - 1s 194us/step - loss: 0.3983 - acc: 0.8356
Epoch 26/50


In [24]:
mean = accuracies.mean()
print('Average accuracy during cross validation: ', mean) 

Average accuracy during cross validation:  0.8358749951049684


In [25]:
accuracies

array([0.845     , 0.83124999, 0.83874999, 0.82749999, 0.86124999,
       0.83125   , 0.82875   , 0.8325    , 0.81125   , 0.85124999])

In [26]:
std = accuracies.std()
print('Accuracy std cross validation: ', std) 

Accuracy std cross validation:  0.013241151807661373


## Parameter tuning

In [27]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [28]:
classifier = KerasClassifier(build_fn = build_classifier)

In [29]:
#define parameter to optimize
#dict key needs to be the same name as defined in classifer
parameters = {'batch_size': [25, 32],
              'epochs': [50, 100, 200],
              'optimizer': ['adam', 'rmsprop']}

In [30]:
grid_search = GridSearchCV(estimator = classifier,
                            param_grid = parameters,
                            scoring = 'accuracy',
                            cv = 10)

In [31]:
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/50
7200/7200 [==============================] - 1s 143us/step - loss: 0.5745 - acc: 0.7949
Epoch 2/50
7200/7200 [==============================] - ETA: 0s - loss: 0.4355 - acc: 0.797 - 0s 60us/step - loss: 0.4351 - acc: 0.7971
Epoch 3/50
7200/7200 [==============================] - 0s 69us/step - loss: 0.4301 - acc: 0.7971
Epoch 4/50
7200/7200 [==============================] - 1s 87us/step - loss: 0.4272 - acc: 0.7971
Epoch 5/50
7200/7200 [==============================] - 1s 99us/step - loss: 0.4249 - acc: 0.7971
Epoch 6/50
7200/7200 [==============================] - 1s 110us/step - loss: 0.4227 - acc: 0.7971
Epoch 7/50
7200/7200 [==============================] - 0s 68us/step - loss: 0.4210 - acc: 0.7971
Epoch 8/50
7200/7200 [==============================] - 1s 92us/step - loss: 0.4191 - acc: 0.8168
Epoch 9/50
7200/7200 [==============================] - 1s 82us/step - loss: 0.4177 - acc: 0.8206
Epoch 10/50
7200/7200 [==============================] - 1s 87us/step - loss: 

7200/7200 [==============================] - 0s 62us/step - loss: 0.4037 - acc: 0.8350
Epoch 34/50
7200/7200 [==============================] - 0s 55us/step - loss: 0.4035 - acc: 0.8353
Epoch 35/50
7200/7200 [==============================] - 1s 100us/step - loss: 0.4033 - acc: 0.8347
Epoch 36/50
7200/7200 [==============================] - 0s 68us/step - loss: 0.4032 - acc: 0.8354
Epoch 37/50
7200/7200 [==============================] - 0s 67us/step - loss: 0.4029 - acc: 0.8344
Epoch 38/50
7200/7200 [==============================] - 0s 66us/step - loss: 0.4026 - acc: 0.8351
Epoch 39/50
7200/7200 [==============================] - 0s 68us/step - loss: 0.4027 - acc: 0.8340
Epoch 40/50
7200/7200 [==============================] - 1s 88us/step - loss: 0.4024 - acc: 0.8347
Epoch 41/50
7200/7200 [==============================] - 1s 70us/step - loss: 0.4025 - acc: 0.8347
Epoch 42/50
7200/7200 [==============================] - 0s 68us/step - loss: 0.4022 - acc: 0.8340
Epoch 43/50
7200/7200

7200/7200 [==============================] - 0s 60us/step - loss: 0.4076 - acc: 0.8347
Epoch 16/50
7200/7200 [==============================] - 0s 62us/step - loss: 0.4072 - acc: 0.8351
Epoch 17/50
7200/7200 [==============================] - 0s 67us/step - loss: 0.4066 - acc: 0.8349
Epoch 18/50
7200/7200 [==============================] - 0s 57us/step - loss: 0.4058 - acc: 0.8361
Epoch 19/50
7200/7200 [==============================] - 0s 59us/step - loss: 0.4057 - acc: 0.8346
Epoch 20/50
7200/7200 [==============================] - 0s 53us/step - loss: 0.4046 - acc: 0.8350
Epoch 21/50
7200/7200 [==============================] - 0s 58us/step - loss: 0.4041 - acc: 0.8337: 0s - loss: 0.3938 - acc: 
Epoch 22/50
7200/7200 [==============================] - 0s 62us/step - loss: 0.4038 - acc: 0.8360
Epoch 23/50
7200/7200 [==============================] - 0s 55us/step - loss: 0.4034 - acc: 0.8360
Epoch 24/50
7200/7200 [==============================] - 0s 55us/step - loss: 0.4032 - acc: 0.

7200/7200 [==============================] - 0s 58us/step - loss: 0.3477 - acc: 0.8587
Epoch 48/50
7200/7200 [==============================] - 0s 53us/step - loss: 0.3470 - acc: 0.8601
Epoch 49/50
7200/7200 [==============================] - 0s 59us/step - loss: 0.3466 - acc: 0.8592
Epoch 50/50
7200/7200 [==============================] - 0s 64us/step - loss: 0.3458 - acc: 0.8597
Epoch 1/50
7200/7200 [==============================] - 1s 174us/step - loss: 0.5510 - acc: 0.7936
Epoch 2/50
7200/7200 [==============================] - 1s 77us/step - loss: 0.4370 - acc: 0.7944
Epoch 3/50
7200/7200 [==============================] - 1s 99us/step - loss: 0.4308 - acc: 0.7944
Epoch 4/50
7200/7200 [==============================] - 1s 121us/step - loss: 0.4275 - acc: 0.7944
Epoch 5/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.4248 - acc: 0.7944
Epoch 6/50
7200/7200 [==============================] - 0s 60us/step - loss: 0.4219 - acc: 0.7944
Epoch 7/50
7200/7200 [===

7200/7200 [==============================] - 0s 65us/step - loss: 0.4060 - acc: 0.8342
Epoch 30/50
7200/7200 [==============================] - 0s 63us/step - loss: 0.4059 - acc: 0.8342
Epoch 31/50
7200/7200 [==============================] - 0s 60us/step - loss: 0.4056 - acc: 0.8354
Epoch 32/50
7200/7200 [==============================] - 1s 78us/step - loss: 0.4054 - acc: 0.8349
Epoch 33/50
7200/7200 [==============================] - 1s 75us/step - loss: 0.4052 - acc: 0.8347
Epoch 34/50
7200/7200 [==============================] - 0s 68us/step - loss: 0.4051 - acc: 0.8346
Epoch 35/50
7200/7200 [==============================] - 0s 63us/step - loss: 0.4044 - acc: 0.8347
Epoch 36/50
7200/7200 [==============================] - 0s 69us/step - loss: 0.4048 - acc: 0.8350
Epoch 37/50
7200/7200 [==============================] - 0s 66us/step - loss: 0.4046 - acc: 0.8358
Epoch 38/50
7200/7200 [==============================] - 1s 90us/step - loss: 0.4044 - acc: 0.8349
Epoch 39/50
7200/7200 

...

7200/7200 [==============================] - 1s 149us/step - loss: 0.3466 - acc: 0.8589
Epoch 18/200
7200/7200 [==============================] - 1s 145us/step - loss: 0.3458 - acc: 0.8608
Epoch 19/200
7200/7200 [==============================] - 1s 122us/step - loss: 0.3450 - acc: 0.8593
Epoch 20/200
7200/7200 [==============================] - 1s 122us/step - loss: 0.3446 - acc: 0.8603
Epoch 21/200
7200/7200 [==============================] - 1s 119us/step - loss: 0.3435 - acc: 0.8596
Epoch 22/200
7200/7200 [==============================] - 1s 122us/step - loss: 0.3438 - acc: 0.8607
Epoch 23/200
7200/7200 [==============================] - 1s 120us/step - loss: 0.3432 - acc: 0.8600
Epoch 24/200
7200/7200 [==============================] - 1s 124us/step - loss: 0.3425 - acc: 0.8601
Epoch 25/200
7200/7200 [==============================] - 1s 121us/step - loss: 0.3424 - acc: 0.8608
Epoch 26/200
7200/7200 [==============================] - 1s 121us/step - loss: 0.3419 - acc: 0.8597
Epo

7200/7200 [==============================] - 1s 123us/step - loss: 0.3342 - acc: 0.8640
Epoch 99/200
7200/7200 [==============================] - 1s 123us/step - loss: 0.3342 - acc: 0.8631
Epoch 100/200
7200/7200 [==============================] - 1s 120us/step - loss: 0.3344 - acc: 0.8651
Epoch 101/200
7200/7200 [==============================] - 1s 132us/step - loss: 0.3341 - acc: 0.8644
Epoch 102/200
7200/7200 [==============================] - 1s 142us/step - loss: 0.3342 - acc: 0.8635
Epoch 103/200
7200/7200 [==============================] - 1s 136us/step - loss: 0.3343 - acc: 0.8626
Epoch 104/200
7200/7200 [==============================] - 1s 141us/step - loss: 0.3340 - acc: 0.8638
Epoch 105/200
7200/7200 [==============================] - 1s 129us/step - loss: 0.3339 - acc: 0.8650
Epoch 106/200
7200/7200 [==============================] - 1s 127us/step - loss: 0.3340 - acc: 0.8633
Epoch 107/200
7200/7200 [==============================] - 1s 123us/step - loss: 0.3340 - acc: 0.

7200/7200 [==============================] - 1s 123us/step - loss: 0.3331 - acc: 0.8642
Epoch 179/200
7200/7200 [==============================] - 1s 125us/step - loss: 0.3332 - acc: 0.8644
Epoch 180/200
7200/7200 [==============================] - 1s 129us/step - loss: 0.3334 - acc: 0.8632
Epoch 181/200
7200/7200 [==============================] - 1s 126us/step - loss: 0.3330 - acc: 0.8638
Epoch 182/200
7200/7200 [==============================] - 1s 124us/step - loss: 0.3333 - acc: 0.8636
Epoch 183/200
7200/7200 [==============================] - 1s 125us/step - loss: 0.3331 - acc: 0.8649
Epoch 184/200
7200/7200 [==============================] - 1s 121us/step - loss: 0.3334 - acc: 0.8647
Epoch 185/200
7200/7200 [==============================] - 1s 123us/step - loss: 0.3331 - acc: 0.8643
Epoch 186/200
7200/7200 [==============================] - 1s 124us/step - loss: 0.3332 - acc: 0.8633
Epoch 187/200
7200/7200 [==============================] - 1s 138us/step - loss: 0.3330 - acc: 0

8000/8000 [==============================] - 1s 128us/step - loss: 0.4021 - acc: 0.8349
Epoch 60/200
8000/8000 [==============================] - 1s 129us/step - loss: 0.4021 - acc: 0.8344
Epoch 61/200
8000/8000 [==============================] - 1s 138us/step - loss: 0.4018 - acc: 0.8357
Epoch 62/200
8000/8000 [==============================] - 1s 123us/step - loss: 0.4017 - acc: 0.8353
Epoch 63/200
8000/8000 [==============================] - 1s 123us/step - loss: 0.4019 - acc: 0.8348
Epoch 64/200
8000/8000 [==============================] - 1s 128us/step - loss: 0.4018 - acc: 0.8341
Epoch 65/200
8000/8000 [==============================] - 1s 125us/step - loss: 0.4015 - acc: 0.8355
Epoch 66/200
8000/8000 [==============================] - 1s 124us/step - loss: 0.4017 - acc: 0.8342
Epoch 67/200
8000/8000 [==============================] - 1s 125us/step - loss: 0.4012 - acc: 0.8339
Epoch 68/200
8000/8000 [==============================] - 1s 123us/step - loss: 0.4015 - acc: 0.8355
Epo

8000/8000 [==============================] - 1s 135us/step - loss: 0.3995 - acc: 0.8346
Epoch 140/200
8000/8000 [==============================] - 1s 137us/step - loss: 0.3993 - acc: 0.8348
Epoch 141/200
8000/8000 [==============================] - 1s 124us/step - loss: 0.3995 - acc: 0.8365
Epoch 142/200
8000/8000 [==============================] - 1s 123us/step - loss: 0.3998 - acc: 0.8359
Epoch 143/200
8000/8000 [==============================] - 1s 124us/step - loss: 0.3994 - acc: 0.8359
Epoch 144/200
8000/8000 [==============================] - 1s 125us/step - loss: 0.3996 - acc: 0.8345
Epoch 145/200
8000/8000 [==============================] - 1s 128us/step - loss: 0.3994 - acc: 0.8354
Epoch 146/200
8000/8000 [==============================] - 1s 128us/step - loss: 0.3991 - acc: 0.8356
Epoch 147/200
8000/8000 [==============================] - 1s 123us/step - loss: 0.3990 - acc: 0.8349
Epoch 148/200
8000/8000 [==============================] - 1s 124us/step - loss: 0.3995 - acc: 0

In [32]:
#get the best parameter and best score

In [33]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [34]:
print('Best parameters are: ', best_parameters)

Best parameters are:  {'batch_size': 32, 'epochs': 200, 'optimizer': 'adam'}


In [35]:
print('Best accuracy are: ', best_accuracy)

Best accuracy are:  0.8445
